In [ ]:
from model import *

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList
import torch

stop_list = ["\n\n", "\n\n\n", "Task:\nBelow"]
# stop_list = ['\nHuman:', '\n```\n']
stop_token_ids = [tokenizer(x, add_special_tokens=False)['input_ids'] for x in stop_list]
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
print(stop_token_ids)

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
from langchain.llms import HuggingFacePipeline

generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=256,  # max number of tokens to generate in the output
    repetition_penalty=1.1,  # without this output begins repeating
    do_sample=True,
    # streamer = transformers.TextStreamer(tokenizer)
)

llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
%load_ext autoreload
%autoreload 2

from prompt_examples.chunked_examples import prompt, prompt_zero, examples


In [ ]:
print(prompt_zero.format(criteria="""Inclusion Criteria:
    •	Adults with a confirmed diagnosis of unresectable, locally advanced and/or metastatic Stage IIIB/IV NSCLC, Stage III/IV PDAC and/or Stage III/IV CRC with no curative-intent treatment options and documented activating KRAS mutation (without known additional actionable driver mutations such as EGFR, ALK or ROS1)
    •	Documented progression and measurable disease after ‚â• 1 prior line of systemic therapy (‚â• 2 and ‚â§ 4 prior lines for NSCLC) with adequate washout period and resolution of treatment-related toxicities to ‚â§ Grade 2
    •	ECOG PS of 0-2 (0-1 for PDAC) and a life expectancy > 3 months in the opinion of the Investigator
    •	Adequate hematological, liver, and renal function
    •	Men and women of childbearing potential must use adequate birth control measures for the duration of the trial and at least 90 days after discontinuing study treatment
    •	Symptomatic and/or untreated CNS or brain metastasis, pre-existing ILD or pericardial/pleural effusion of ‚â• grade 2 or requiring chronic oxygen therapy for COPD or pleural effusions
    •	Serious concomitant disorder including infection
    •	Known positive test for HIV, HCV, HBV surface antigen

    Exclusion Criteria:
    •	Concurrent malignancy in the previous 2 years
    •	Prior menin inhibitor therapy
    •	Requiring treatment with a strong or moderate CYP3A inhibitor/inducer
    •	Significant cardiovascular disease or QTcF or QTcB prolongation.
    •	Major surgery within 4 weeks prior to first dose
    •	Women who are pregnant or lactating.
"""))

In [ ]:
from token_counting import *
globalize_token_metrics(examples)
print('  avg prompt:', AVG_PROMPT_LEN)
print('  min prompt:', MIN_PROMPT_LEN)
print('  max prompt:', MAX_PROMPT_LEN)
print('avg response:', AVG_RES_LEN)

In [ ]:
from chunking import *

In [ ]:
from langchain.chains import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt)
# llm_chain_zero = LLMChain(llm=llm, prompt=prompt_zero)

In [ ]:
import langchain
langchain.debug = True
langchain.verbose = True

n = '01'
folder = f'test_results/trial{n}'

original_doc = parse_file(f'{folder}/unstructured_ec.txt')
doc_chunks = chunk_ec(original_doc, min_criterions_per_section=3, target_input_size=0.6)
inc = []
exc = []

for idx, doc in enumerate(doc_chunks):
    results = llm_chain.invoke(input={'criteria': str(doc)})
    with open(f'{folder}/smaller_chunking_task_8bit_{idx}_output.txt', 'w', encoding='utf-8') as fileout:
        fileout.write(results['text'])

    for line in doc.inc:
        if line not in inc:
            inc.append(line)
    for line in doc.exc:
        if line not in exc:
            exc.append(line)

# reassembled_doc = ECDoc(inc, exc)

# with open(f'{folder}/smaller_chunking_reassembled.txt', 'w') as docfile:
#     docfile.write(str(reassembled_doc))

In [ ]:
# Copy the prompts that Langchain logs
prompts_used = [
    "You are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria:\n    •\tAdults with a confirmed diagnosis of unresectable, locally advanced and/or metastatic Stage IIIB/IV NSCLC, Stage III/IV PDAC and/or Stage III/IV CRC with no curative-intent treatment options and documented activating KRAS mutation (without known additional actionable driver mutations such as EGFR, ALK or ROS1)\n    •\tDocumented progression and measurable disease after ‚â• 1 prior line of systemic therapy (‚â• 2 and ‚â§ 4 prior lines for NSCLC) with adequate washout period and resolution of treatment-related toxicities to ‚â§ Grade 2\n    •\tECOG PS of 0-2 (0-1 for PDAC) and a life expectancy > 3 months in the opinion of the Investigator\n    •\tAdequate hematological, liver, and renal function\n    •\tMen and women of childbearing potential must use adequate birth control measures for the duration of the trial and at least 90 days after discontinuing study treatment\n    •\tSymptomatic and/or untreated CNS or brain metastasis, pre-existing ILD or pericardial/pleural effusion of ‚â• grade 2 or requiring chronic oxygen therapy for COPD or pleural effusions\n    •\tSerious concomitant disorder including infection\n    •\tKnown positive test for HIV, HCV, HBV surface antigen\n\n    Exclusion Criteria:\n    •\tConcurrent malignancy in the previous 2 years\n    •\tPrior menin inhibitor therapy\n    •\tRequiring treatment with a strong or moderate CYP3A inhibitor/inducer\n    •\tSignificant cardiovascular disease or QTcF or QTcB prolongation.\n    •\tMajor surgery within 4 weeks prior to first dose\n    •\tWomen who are pregnant or lactating.\n\n| Type | Original Text | Disease/Condition | Procedure | Drug | Biomarker | Computable Rule |\n| --- | --- | --- | --- | --- | --- | --- |\n| Inclusion | Adults with a confirmed diagnosis of unresectable, locally advanced and/or metastatic Stage IIIB/IV NSCLC, Stage III/IV PDAC and/or Stage III/IV CRC with no curative-intent treatment options and documented activating KRAS mutation (without known additional actionable driver mutations such as EGFR, ALK or ROS1) | Metastatic Lung Non-Small Cell Carcinoma, Metastatic Pancreatic Ductal Adenocarcinoma, Metastatic Colorectal Carcinoma | | | KRAS | diagnosis == \"Metastatic Lung Non-Small Cell Carcinoma\" OR diagnosis == \"Metastatic Pancreatic Ductal Adenocarcinoma\" OR diagnosis == \"Metastatic Colorectal Carcinoma\", KRAS is True |\n| Exclusion | Prior menin inhibitor therapy | | | Menin inhibitor | | Menin inhibitor is True |\n\n\nYou are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria\n•\tPatients must have undergone complete surgical resection of their stage IIA, IIB, IIIA or IIIB non-squamous or squamous b NSCLC per American Joint Committee on Cancer (AJCC) 8th edition and have had negative margins. N3 disease is not allowed.  \n•\tBaseline chest computed tomography (CT) with or without contrast must be performed within 6 months (180 days) prior to randomization to ensure no evidence of disease; if clinically indicated additional imaging studies must be performed to rule out metastatic disease\n•\tEastern Cooperative Oncology Group (ECOG) performance status 0 or 1\n\nExclusion Criteria\n•\tNo known interstitial fibrosis or interstitial lung disease\n•\tNo prior treatment with crizotinib or another ALK inhibitor",
    "You are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria:\n    •\tAdults with a confirmed diagnosis of unresectable, locally advanced and/or metastatic Stage IIIB/IV NSCLC, Stage III/IV PDAC and/or Stage III/IV CRC with no curative-intent treatment options and documented activating KRAS mutation (without known additional actionable driver mutations such as EGFR, ALK or ROS1)\n    •\tDocumented progression and measurable disease after ‚â• 1 prior line of systemic therapy (‚â• 2 and ‚â§ 4 prior lines for NSCLC) with adequate washout period and resolution of treatment-related toxicities to ‚â§ Grade 2\n    •\tECOG PS of 0-2 (0-1 for PDAC) and a life expectancy > 3 months in the opinion of the Investigator\n    •\tAdequate hematological, liver, and renal function\n    •\tMen and women of childbearing potential must use adequate birth control measures for the duration of the trial and at least 90 days after discontinuing study treatment\n    •\tSymptomatic and/or untreated CNS or brain metastasis, pre-existing ILD or pericardial/pleural effusion of ‚â• grade 2 or requiring chronic oxygen therapy for COPD or pleural effusions\n    •\tSerious concomitant disorder including infection\n    •\tKnown positive test for HIV, HCV, HBV surface antigen\n\n    Exclusion Criteria:\n    •\tConcurrent malignancy in the previous 2 years\n    •\tPrior menin inhibitor therapy\n    •\tRequiring treatment with a strong or moderate CYP3A inhibitor/inducer\n    •\tSignificant cardiovascular disease or QTcF or QTcB prolongation.\n    •\tMajor surgery within 4 weeks prior to first dose\n    •\tWomen who are pregnant or lactating.\n\n| Type | Original Text | Disease/Condition | Procedure | Drug | Biomarker | Computable Rule |\n| --- | --- | --- | --- | --- | --- | --- |\n| Inclusion | Adults with a confirmed diagnosis of unresectable, locally advanced and/or metastatic Stage IIIB/IV NSCLC, Stage III/IV PDAC and/or Stage III/IV CRC with no curative-intent treatment options and documented activating KRAS mutation (without known additional actionable driver mutations such as EGFR, ALK or ROS1) | Metastatic Lung Non-Small Cell Carcinoma, Metastatic Pancreatic Ductal Adenocarcinoma, Metastatic Colorectal Carcinoma | | | KRAS | diagnosis == \"Metastatic Lung Non-Small Cell Carcinoma\" OR diagnosis == \"Metastatic Pancreatic Ductal Adenocarcinoma\" OR diagnosis == \"Metastatic Colorectal Carcinoma\", KRAS is True |\n| Exclusion | Prior menin inhibitor therapy | | | Menin inhibitor | | Menin inhibitor is True |\n\n\nYou are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria\n•\tPatients must be registered to the ALCHEMIST-SCREEN (ALLIANCE A151216) trial prior to randomization \n•\tPositive for translocation or inversion events involving the ALK gene locus (e.g. resulting in EML4-ALK fusion) as defined by a Clinical Laboratory Improvement Act (CLIA)-approved test including: (1) translocation or inversion events involving the ALK gene locus (e.g. resulting in EML4-ALK fusion) as determined by the Vysis Break Point fluorescence in situ hybridization (FISH) assay; (2) ALK protein expression by immunohistochemistry (IHC); or (3) ALK rearrangement identified by next generation (NextGen) sequencing; this must have been performed:\no\tBy a local CLIA certified laboratory: Report must indicate the results as well as the CLIA number of the laboratory which performed the assay; tissue must be available for submission for central, retrospective confirmation of the ALK fusion status via ALCHEMIST-SCREEN (ALLIANCE A151216); OR\n\nExclusion Criteria\n•\tNo known interstitial fibrosis or interstitial lung disease\n•\tNo prior treatment with crizotinib or another ALK inhibitor",
    "You are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria\n    •\tParticipants must be ‚â• 18 years of age\n    •\tHistologically or cytologically confirmed diagnosis of metastatic solid tumors\n    •\tEastern Cooperative Oncology Group (ECOG) performance status 0-1\n    •\tAll participants should have at least 1 measurable disease per RECIST v1.1. An irradiated lesion can be considered measurable only if progression has been demonstrated on the irradiated lesion.\n    •\tBody weight within [45 - 150 kg] (inclusive)\n    •\tAll Contraceptive use by men and women should be consistent with local regulations regarding the methods of contraception for those participating in clinical studies.\n    •\tCapable of giving signed informed consent\n    •\tAny clinically significant cardiac disease\n    •\tHistory of or current interstitial lung disease or pneumonitis\n\n    Exclusion Criteria\n    •\tUncontrolled or unresolved acute renal failure\n    •\tPrior solid organ or hematologic transplant.\n    •\tKnown positivity with human immunodeficiency virus (HIV), known active hepatitis A, B, and C, or uncontrolled chronic or ongoing infectious requiring parenteral treatment.\n    •\tReceipt of a live-virus vaccination within 28 days of planned treatment start\n    •\tParticipation in a concurrent clinical study in the treatment period.\n    •\tInadequate hematologic, hepatic and renal function\n    •\tParticipant not suitable for participation, whatever the reason, as judged by the Investigator, including medical or clinical conditions.\n\n| Type | Original Text | Disease/Condition | Procedure | Drug | Biomarker | Computable Rule |\n| --- | --- | --- | --- | --- | --- | --- |\n| Inclusion | Histologically or cytologically confirmed diagnosis of metastatic solid tumors | Metastatic solid tumor | | | | diagnosis == \"Metastatic solid tumor\" |\n| Exclusion | Prior solid organ or hematologic transplant. | | Solid organ transplantation | | | Solid organ transplantation is True |\n\n\nYou are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria\no\tPatient registered to and the ALK fusion status performed centrally on the ALCHEMIST-SCREEN (ALLIANCE A151216)\n•\tAll females of childbearing potential must have a blood or urine pregnancy test within 72 hours prior to randomization to rule out pregnancy; a female of childbearing potential is any woman, regardless of sexual orientation or whether they have undergone tubal ligation, who meets the following criteria: 1) has not undergone a hysterectomy or bilateral oophorectomy; or 2) has not been naturally postmenopausal for at least 24 consecutive months (i.e., has had menses at any time in the preceding 24 consecutive months)\n•\tWomen of childbearing potential and sexually active males must be strongly advised to practice abstinence or use an accepted and effective method of contraception\n\nExclusion Criteria\n•\tNo ongoing cardiac dysrhythmias of grade >= 2 National Cancer Institute (NCI) Common Terminology Criteria for Adverse Events (CTCAE) version 4.0, uncontrolled atrial fibrillation (any grade), or corrected QT (QTc) interval > 470 msec\n•\tNo use of medications, herbals, or foods that are known potent cytochrome P450, subfamily 3A, polypeptide 4 (CYP3A4) inhibitors or inducers, included but not limited to those outlined",
    "You are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria\n    •\tParticipants must be ‚â• 18 years of age\n    •\tHistologically or cytologically confirmed diagnosis of metastatic solid tumors\n    •\tEastern Cooperative Oncology Group (ECOG) performance status 0-1\n    •\tAll participants should have at least 1 measurable disease per RECIST v1.1. An irradiated lesion can be considered measurable only if progression has been demonstrated on the irradiated lesion.\n    •\tBody weight within [45 - 150 kg] (inclusive)\n    •\tAll Contraceptive use by men and women should be consistent with local regulations regarding the methods of contraception for those participating in clinical studies.\n    •\tCapable of giving signed informed consent\n    •\tAny clinically significant cardiac disease\n    •\tHistory of or current interstitial lung disease or pneumonitis\n\n    Exclusion Criteria\n    •\tUncontrolled or unresolved acute renal failure\n    •\tPrior solid organ or hematologic transplant.\n    •\tKnown positivity with human immunodeficiency virus (HIV), known active hepatitis A, B, and C, or uncontrolled chronic or ongoing infectious requiring parenteral treatment.\n    •\tReceipt of a live-virus vaccination within 28 days of planned treatment start\n    •\tParticipation in a concurrent clinical study in the treatment period.\n    •\tInadequate hematologic, hepatic and renal function\n    •\tParticipant not suitable for participation, whatever the reason, as judged by the Investigator, including medical or clinical conditions.\n\n| Type | Original Text | Disease/Condition | Procedure | Drug | Biomarker | Computable Rule |\n| --- | --- | --- | --- | --- | --- | --- |\n| Inclusion | Histologically or cytologically confirmed diagnosis of metastatic solid tumors | Metastatic solid tumor | | | | diagnosis == \"Metastatic solid tumor\" |\n| Exclusion | Prior solid organ or hematologic transplant. | | Solid organ transplantation | | | Solid organ transplantation is True |\n\n\nYou are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria\n•\tPatients must be adequately recovered from surgery at the time of randomization\n•\tThe minimum time requirement between date of surgery and randomization must be at least 4 weeks (28 days)\n•\tThe maximum time requirement between surgery and randomization must be:\no\t4 months (120 days) if no adjuvant chemotherapy was administered\n\nExclusion Criteria\n•\tNo ongoing cardiac dysrhythmias of grade >= 2 National Cancer Institute (NCI) Common Terminology Criteria for Adverse Events (CTCAE) version 4.0, uncontrolled atrial fibrillation (any grade), or corrected QT (QTc) interval > 470 msec\n•\tNo use of medications, herbals, or foods that are known potent cytochrome P450, subfamily 3A, polypeptide 4 (CYP3A4) inhibitors or inducers, included but not limited to those outlined",
    "You are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria\n    •\tParticipants must be ‚â• 18 years of age\n    •\tHistologically or cytologically confirmed diagnosis of metastatic solid tumors\n    •\tEastern Cooperative Oncology Group (ECOG) performance status 0-1\n    •\tAll participants should have at least 1 measurable disease per RECIST v1.1. An irradiated lesion can be considered measurable only if progression has been demonstrated on the irradiated lesion.\n    •\tBody weight within [45 - 150 kg] (inclusive)\n    •\tAll Contraceptive use by men and women should be consistent with local regulations regarding the methods of contraception for those participating in clinical studies.\n    •\tCapable of giving signed informed consent\n    •\tAny clinically significant cardiac disease\n    •\tHistory of or current interstitial lung disease or pneumonitis\n\n    Exclusion Criteria\n    •\tUncontrolled or unresolved acute renal failure\n    •\tPrior solid organ or hematologic transplant.\n    •\tKnown positivity with human immunodeficiency virus (HIV), known active hepatitis A, B, and C, or uncontrolled chronic or ongoing infectious requiring parenteral treatment.\n    •\tReceipt of a live-virus vaccination within 28 days of planned treatment start\n    •\tParticipation in a concurrent clinical study in the treatment period.\n    •\tInadequate hematologic, hepatic and renal function\n    •\tParticipant not suitable for participation, whatever the reason, as judged by the Investigator, including medical or clinical conditions.\n\n| Type | Original Text | Disease/Condition | Procedure | Drug | Biomarker | Computable Rule |\n| --- | --- | --- | --- | --- | --- | --- |\n| Inclusion | Histologically or cytologically confirmed diagnosis of metastatic solid tumors | Metastatic solid tumor | | | | diagnosis == \"Metastatic solid tumor\" |\n| Exclusion | Prior solid organ or hematologic transplant. | | Solid organ transplantation | | | Solid organ transplantation is True |\n\n\nYou are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria\no\t9 months (270 days) if adjuvant chemotherapy was administered\no\t11 months (330 days) if adjuvant chemotherapy and radiation therapy were administered\n•\tPatients must have completed any prior adjuvant chemotherapy or radiation therapy 2 or more weeks (6 or more weeks for mitomycin and nitrosoureas) prior to randomization and be adequately recovered at the time of randomization\n\nExclusion Criteria\n•\tPatients may not be receiving any other investigational agents while on study\n•\tPatients must NOT have uncontrolled intercurrent illness including, but not limited to, serious ongoing or active infection, symptomatic congestive heart failure, unstable angina pectoris, uncontrolled cardiac arrhythmia, or psychiatric illness/social situations that would limit compliance with study requirements",
    "You are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria:\n    •\tAdults with a confirmed diagnosis of unresectable, locally advanced and/or metastatic Stage IIIB/IV NSCLC, Stage III/IV PDAC and/or Stage III/IV CRC with no curative-intent treatment options and documented activating KRAS mutation (without known additional actionable driver mutations such as EGFR, ALK or ROS1)\n    •\tDocumented progression and measurable disease after ‚â• 1 prior line of systemic therapy (‚â• 2 and ‚â§ 4 prior lines for NSCLC) with adequate washout period and resolution of treatment-related toxicities to ‚â§ Grade 2\n    •\tECOG PS of 0-2 (0-1 for PDAC) and a life expectancy > 3 months in the opinion of the Investigator\n    •\tAdequate hematological, liver, and renal function\n    •\tMen and women of childbearing potential must use adequate birth control measures for the duration of the trial and at least 90 days after discontinuing study treatment\n    •\tSymptomatic and/or untreated CNS or brain metastasis, pre-existing ILD or pericardial/pleural effusion of ‚â• grade 2 or requiring chronic oxygen therapy for COPD or pleural effusions\n    •\tSerious concomitant disorder including infection\n    •\tKnown positive test for HIV, HCV, HBV surface antigen\n\n    Exclusion Criteria:\n    •\tConcurrent malignancy in the previous 2 years\n    •\tPrior menin inhibitor therapy\n    •\tRequiring treatment with a strong or moderate CYP3A inhibitor/inducer\n    •\tSignificant cardiovascular disease or QTcF or QTcB prolongation.\n    •\tMajor surgery within 4 weeks prior to first dose\n    •\tWomen who are pregnant or lactating.\n\n| Type | Original Text | Disease/Condition | Procedure | Drug | Biomarker | Computable Rule |\n| --- | --- | --- | --- | --- | --- | --- |\n| Inclusion | Adults with a confirmed diagnosis of unresectable, locally advanced and/or metastatic Stage IIIB/IV NSCLC, Stage III/IV PDAC and/or Stage III/IV CRC with no curative-intent treatment options and documented activating KRAS mutation (without known additional actionable driver mutations such as EGFR, ALK or ROS1) | Metastatic Lung Non-Small Cell Carcinoma, Metastatic Pancreatic Ductal Adenocarcinoma, Metastatic Colorectal Carcinoma | | | KRAS | diagnosis == \"Metastatic Lung Non-Small Cell Carcinoma\" OR diagnosis == \"Metastatic Pancreatic Ductal Adenocarcinoma\" OR diagnosis == \"Metastatic Colorectal Carcinoma\", KRAS is True |\n| Exclusion | Prior menin inhibitor therapy | | | Menin inhibitor | | Menin inhibitor is True |\n\n\nYou are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria\nNOTE: Patients taking low dose methotrexate for non-malignant conditions and other cytotoxic agents for non-malignant conditions are allowed to continue treatment while on study\nNOTE: Neo-adjuvant chemotherapy or radiation therapy for the resected lung cancer is not permitted (inclusion--- t) | \n•\tPatients must have adequate organ function as defined by the following criteria within 2 weeks prior to randomization:\nNOTE: it is strongly encouraged that these tests take place no more than one week prior to randomization to meet the 2 week requirement for randomization\n\nExclusion Criteria\n•\tPatients may not be receiving any other investigational agents while on study\n•\tPatients must NOT have uncontrolled intercurrent illness including, but not limited to, serious ongoing or active infection, symptomatic congestive heart failure, unstable angina pectoris, uncontrolled cardiac arrhythmia, or psychiatric illness/social situations that would limit compliance with study requirements",
    "You are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria:\n    •\tAdults with a confirmed diagnosis of unresectable, locally advanced and/or metastatic Stage IIIB/IV NSCLC, Stage III/IV PDAC and/or Stage III/IV CRC with no curative-intent treatment options and documented activating KRAS mutation (without known additional actionable driver mutations such as EGFR, ALK or ROS1)\n    •\tDocumented progression and measurable disease after ‚â• 1 prior line of systemic therapy (‚â• 2 and ‚â§ 4 prior lines for NSCLC) with adequate washout period and resolution of treatment-related toxicities to ‚â§ Grade 2\n    •\tECOG PS of 0-2 (0-1 for PDAC) and a life expectancy > 3 months in the opinion of the Investigator\n    •\tAdequate hematological, liver, and renal function\n    •\tMen and women of childbearing potential must use adequate birth control measures for the duration of the trial and at least 90 days after discontinuing study treatment\n    •\tSymptomatic and/or untreated CNS or brain metastasis, pre-existing ILD or pericardial/pleural effusion of ‚â• grade 2 or requiring chronic oxygen therapy for COPD or pleural effusions\n    •\tSerious concomitant disorder including infection\n    •\tKnown positive test for HIV, HCV, HBV surface antigen\n\n    Exclusion Criteria:\n    •\tConcurrent malignancy in the previous 2 years\n    •\tPrior menin inhibitor therapy\n    •\tRequiring treatment with a strong or moderate CYP3A inhibitor/inducer\n    •\tSignificant cardiovascular disease or QTcF or QTcB prolongation.\n    •\tMajor surgery within 4 weeks prior to first dose\n    •\tWomen who are pregnant or lactating.\n\n| Type | Original Text | Disease/Condition | Procedure | Drug | Biomarker | Computable Rule |\n| --- | --- | --- | --- | --- | --- | --- |\n| Inclusion | Adults with a confirmed diagnosis of unresectable, locally advanced and/or metastatic Stage IIIB/IV NSCLC, Stage III/IV PDAC and/or Stage III/IV CRC with no curative-intent treatment options and documented activating KRAS mutation (without known additional actionable driver mutations such as EGFR, ALK or ROS1) | Metastatic Lung Non-Small Cell Carcinoma, Metastatic Pancreatic Ductal Adenocarcinoma, Metastatic Colorectal Carcinoma | | | KRAS | diagnosis == \"Metastatic Lung Non-Small Cell Carcinoma\" OR diagnosis == \"Metastatic Pancreatic Ductal Adenocarcinoma\" OR diagnosis == \"Metastatic Colorectal Carcinoma\", KRAS is True |\n| Exclusion | Prior menin inhibitor therapy | | | Menin inhibitor | | Menin inhibitor is True |\n\n\nYou are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria\no\tSerum aspartate aminotransferase (AST) and serum alanine aminotransferase (ALT) =< 2.5 x upper limit of normal (ULN) \no\tTotal serum bilirubin =< 1.5 x ULN\no\tAbsolute neutrophil count (ANC) >= 1500/mm^3\n\nExclusion Criteria\n•\tWomen must not be pregnant or breast-feeding because, based on the mechanism of action, crizotinib may cause fetal harm when administered during pregnancy; in animal studies, teratogenicity was not evident, but embryotoxic and fetotoxic effects were noted in rats at crizotinib exposures similar to and above those observed in humans at the recommended clinical dose\n•\tPatients must not have any history of locally advanced or metastatic cancer requiring systemic therapy within 5 years from randomization, with the exception of in-situ carcinomas and non-melanoma skin cancer; patients must have no previous primary lung cancer diagnosed concurrently or within the past 2 years",
    "You are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria:\n    •\tAdults with a confirmed diagnosis of unresectable, locally advanced and/or metastatic Stage IIIB/IV NSCLC, Stage III/IV PDAC and/or Stage III/IV CRC with no curative-intent treatment options and documented activating KRAS mutation (without known additional actionable driver mutations such as EGFR, ALK or ROS1)\n    •\tDocumented progression and measurable disease after ‚â• 1 prior line of systemic therapy (‚â• 2 and ‚â§ 4 prior lines for NSCLC) with adequate washout period and resolution of treatment-related toxicities to ‚â§ Grade 2\n    •\tECOG PS of 0-2 (0-1 for PDAC) and a life expectancy > 3 months in the opinion of the Investigator\n    •\tAdequate hematological, liver, and renal function\n    •\tMen and women of childbearing potential must use adequate birth control measures for the duration of the trial and at least 90 days after discontinuing study treatment\n    •\tSymptomatic and/or untreated CNS or brain metastasis, pre-existing ILD or pericardial/pleural effusion of ‚â• grade 2 or requiring chronic oxygen therapy for COPD or pleural effusions\n    •\tSerious concomitant disorder including infection\n    •\tKnown positive test for HIV, HCV, HBV surface antigen\n\n    Exclusion Criteria:\n    •\tConcurrent malignancy in the previous 2 years\n    •\tPrior menin inhibitor therapy\n    •\tRequiring treatment with a strong or moderate CYP3A inhibitor/inducer\n    •\tSignificant cardiovascular disease or QTcF or QTcB prolongation.\n    •\tMajor surgery within 4 weeks prior to first dose\n    •\tWomen who are pregnant or lactating.\n\n| Type | Original Text | Disease/Condition | Procedure | Drug | Biomarker | Computable Rule |\n| --- | --- | --- | --- | --- | --- | --- |\n| Inclusion | Adults with a confirmed diagnosis of unresectable, locally advanced and/or metastatic Stage IIIB/IV NSCLC, Stage III/IV PDAC and/or Stage III/IV CRC with no curative-intent treatment options and documented activating KRAS mutation (without known additional actionable driver mutations such as EGFR, ALK or ROS1) | Metastatic Lung Non-Small Cell Carcinoma, Metastatic Pancreatic Ductal Adenocarcinoma, Metastatic Colorectal Carcinoma | | | KRAS | diagnosis == \"Metastatic Lung Non-Small Cell Carcinoma\" OR diagnosis == \"Metastatic Pancreatic Ductal Adenocarcinoma\" OR diagnosis == \"Metastatic Colorectal Carcinoma\", KRAS is True |\n| Exclusion | Prior menin inhibitor therapy | | | Menin inhibitor | | Menin inhibitor is True |\n\n\nYou are in the role of an abstractor who will analyze eligibility criteria for a clinical trial and represent the information as a list of individual criteria in a tabular format that will contain the following columns: \nType: listing whether criterion is an Exclusion or Inclusion criterion\nOriginal Text: the original text of the criterion\nDisease/Condition: If the criterion contains a disease or condition name it by its canonical name\nProcedure: If the criterion contains a therapeutic procedure name it by its canonical name\nDrug:  If the criterion contains a therapeutic drug name it by its canonical name\nBiomarker:  If the criterion contains a biomarker name it by its canonical name\nComputable Rule: Translate the criteria into a logical expression that could be interpreted programmatically\n    Inclusion Criteria\no\tPlatelets >= 30,000/mm^3\no\tHemoglobin >= 8.0 g/dL\no\tSerum creatinine =< 2 x ULN\n•\tPrior to randomization patients with any non-hematologic toxicity from surgery, chemotherapy, or radiation must have recovered to grade =< 1 with the exception of alopecia and the criteria outlined\n\nExclusion Criteria\n•\tWomen must not be pregnant or breast-feeding because, based on the mechanism of action, crizotinib may cause fetal harm when administered during pregnancy; in animal studies, teratogenicity was not evident, but embryotoxic and fetotoxic effects were noted in rats at crizotinib exposures similar to and above those observed in humans at the recommended clinical dose\n•\tPatients must not have any history of locally advanced or metastatic cancer requiring systemic therapy within 5 years from randomization, with the exception of in-situ carcinomas and non-melanoma skin cancer; patients must have no previous primary lung cancer diagnosed concurrently or within the past 2 years"
]

for idx, prompt_used in enumerate(prompts_used):
    prompt_len = get_token_len(prompt_used)
    with open(f'{folder}/repeat_up_task_{idx}_prompt.txt', 'w', encoding='utf-8') as fileout:
        fileout.write("""[metrics]
length={prompt_len}

[prompt]
{prompt}""".format(prompt=prompt_used, prompt_len=prompt_len))